# Poke-Gan

![Pokemon logo](./assets/img/pokemon-logo.png "Pokemon logo")

# Motivacija

Rekreiranje rada pix2pix na nekom proizvoljnom problemu. Primarni cilj je bio dublje upoznavanje sa generativnim suparničkim mrežama.

Ideja: https://affinelayer.com/pixsrv/

Rad: https://arxiv.org/abs/1611.07004 <br/>
Implementacija je takođe dostupna: https://github.com/phillipi/pix2pix

# Podaci

Skup podataka je preuzet sa *Kaggle*: https://www.kaggle.com/kvpratama/pokemon-images-dataset

![Pokemons from dataset](./assets/img/dataset_example.png "Dataset example")

Veličina skupa podataka je 819 slika. Iz tog skupa odvojena je 41 slika za validacioni skup, dok je ostatak korišćen za treniranje.

## Generisanje ivica

Nad originalnim skupom podataka primenjeni su filteri za generisanje ivica:
* Canny edge - Jednostavno pronalaženje ivica
* ApproxPolyDP - Pronalaženje ivica pri čemu su one grublje i više liče na crtež rukom

<img src="./assets/img/edge_sketch_example.png" width="800"/>

Problem kod *ApproxPolyDP* jeste pronalaženje odgovarajućeg praga za sve slike. Naime, taj prag određuje nivo detalja na slici koji nije isti za sve crteže. Iz ovog razloga vidimo da je slika u prvom redu značajno lošije predstavljna nego slika ispod nje. Moguće unaprađenje alogritma bi bilo da se prag odredi za svaku sliku ponaosob.

# Metod

Kao **generator** korišćena je modifikacija [UNet](https://arxiv.org/abs/1505.04597) mreže. <br/>
Kao **diskriminator** korišćen je PatchGAN.

![Model architecture](./assets/img/pix2pix_model.png "Model architecture")

Generarator na ulažu dobija sliku pokemona i na izlazu daje obojen crtež. Umesto dodavanja šuma sa strane, model ostaje u stanju treninga (nema izbacivanja *Dropout* i *Batch normalization* slojeva) što se smatra dovoljnim.

## Funkcije greške

Mogu se videti u radu na 3. strani. <br/>
Za ovaj projekat od velikog je zančaja faktor λ koji množi vrednost L1 u funkciji greške kod generatora.

# Trening

Kako bi se ispitalo što više kombinacija hiperparametara korišćen je alat [wandb](https://wandb.ai/).
Ovaj alat prima na ulazu moguće vrednosti hiperparametara i onda pokreće trening za neke od kombinacija. Pored toga pruža i lepu vizuelizaciju koja se može videti u nastavku.

### Treniranje

Nakon nekoliko ručno ispitanih kombinacija parametara uočena je važnost parametara lambda. Ovo je i logično jer, kao što je u radu opisano, greška l1 (koju lambda množi) tera izlaz iz generatora da liči na ciljnu sliku.

Prilikom prvog pokretanja alata wandb, broj epoha je fiksiran na 50. Taj broj izabran je empirijski jer do tada dobri modeli nauče da boje, dok lošiji daju neprihvatljive izlaze, a i vreme treniranja je prihvatljivo. Nakon ovoga mogu se uočiti kombinacije parametara koje daju lošije rezultate.

Moguće vrednosti parametara: <br/>
lambda ∈ {50, 100, 150, 200} <br/>
batch size ∈ {32, 64} <br/>
learning rate ∈ {1e-5, 1e-4, 2e-4, 3e-4} <br/>

<br/>

![wandb table for sweep 1](./assets/img/sweep-1.png "Training results for sweep 1")

<br/>
<br/>

#### Parallel coordinates visualization

![parallel coords visualization for sweep 1](./assets/img/parallel-coords-sweep-1.png "Parallel coords visualization for sweep 1")

#### Greška generatora i diskriminatora

Napomena: Smoothing je podešen na 0.99

![generator and discriminator losses for sweep 1](./assets/img/losses-sweep-1.png "Generator and discriminator losses for sweep 1")


#### Greška generatora na validacionom skupu
![generator validation loss for sweep 1](./assets/img/val-loss-sweep-1.png "Generator validation loss for sweep 1")


#### Ocena važnosti parametara

![parameter importance for sweep 1](./assets/img/param-importance-train-sweep-1.png "Parameter importance for sweep 1")


### Rezulati

Obojen ručno crtan crtež koji se ne nalazi ni u skupu za treniranje, ni u skupu za validaciju.

![Results for sweep 1](./assets/img/custom-sketch-sweep-1.png "Result: Painted custom sketch")

### Zaključak

Izbor parametra lambda retko je bio 100, a on se odokativno čini jako dobrim. Kao što vidimo parametar *batch size* nije toliko relevantan te je on fiksiran na 64.

### Ponovljeno treniranje

Nakon izučavanja vrednosti hiperparametara i njhove važnosti trening je pokrenut ponovo. Parametar *batch size* fiksiran je na 64 i moguće vrednosti lambda su oko 100. Ovga puta trenira se 100 epoha.

Moguće vrednosti parametara: <br/>
lambda ∈ {80, 90, 100, 110, 120, 130} <br/>
batch size ∈ {64} <br/>
learning rate ∈ {1e-5, 1e-4, 2e-4, 3e-4} <br/>

<br/>

![wandb table for sweep 2](./assets/img/sweep-2.png "Training results for sweep 2")

<br/>
<br/>

#### Greška generatora i diskriminatora

Napomena: Smoothing je podešen na 0.99

![generator and discriminator losses for sweep 2](./assets/img/losses-sweep-2.png "Generator and discriminator losses for sweep 2")


#### Ocena važnosti parametara

![parameter importance for sweep 2](./assets/img/param-importance-train-sweep-2.png "Parameter importance for sweep 2")

### Rezulati

Obojen ručno crtan crtež koji se ne nalazi ni u skupu za treniranje, ni u skupu za validaciju.

![Results for sweep 2](./assets/img/custom-sketch-sweep-2.png "Result: Painted custom sketch")


### Parallel coordinates visualization 

Sve uzete vrednosti parametara i njihov uticaj na ciljnu promenljivu:

![parallel coords visualization](./assets/img/parallel-coords.png "Parallel coords visualization")


# Isprobajte demo!

Neki od ovih modela se nalaze u trained_models folderu i mogu se isprobati pokretenjem demo jupyter sveske.